# Build Junctions from Nodes

In [1]:
import random
from collections import defaultdict

import numpy as np
import geopandas as gpd
import folium

# Map styling configuration
MAP_COLORS = ["red", "blue", "green", "orange", "purple", "pink", "yellow"]
SEGMENT_STYLE = {"weight": 5, "opacity": 0.5}
ACCIDENT_MARKER_STYLE = {"radius": 1, "fill": True, "fill_opacity": 1.0}
CROSSING_MARKER_STYLE = {"radius": 5, "color": "black", "fill": True, "fill_color": "black", "fill_opacity": 1.0}

In [2]:
# this is the joined_unique dataframe containing all accidents with the assigned strava segment
accidents_with_segment = gpd.read_parquet("../data/dataframes/accidents_with_segment.parquet")

accident_locations = gpd.read_parquet("../data/dataframes/accident_locations.parquet")
strava_segments = gpd.read_parquet("../data/dataframes/strava_segments.parquet")
strava_segments['coord_list'] = strava_segments.geometry.apply(lambda line: list(line.coords))

In [3]:
# 1. Map every coordinate to the segments that pass through it
coords_to_segments = defaultdict(set)

for _, row in strava_segments.iterrows():
    name = row["counter_name"]
    # Using a set to avoid counting the same segment more than once
    for coord in row.coord_list:
        coords_to_segments[coord].add(name)

# 2. Filter the dictionary for "crossings" (more than 1 unique segment)
all_crossings = {
    coord: segments 
    for coord, segments in coords_to_segments.items()
    if len(segments) > 1
}

In [4]:
def _calculate_bbox(coordinates, lat_tol, lon_tol):
    """Calculate bounding box from center coordinates and tolerances."""
    min_lat = coordinates[0] - lat_tol
    max_lat = coordinates[0] + lat_tol
    min_lon = coordinates[1] - lon_tol
    max_lon = coordinates[1] + lon_tol
    return min_lat, max_lat, min_lon, max_lon


def _create_base_map(coordinates, zoom):
    """Create and return a folium map with CartoDB light tiles."""
    m = folium.Map(location=coordinates, zoom_start=zoom, tiles=None)
    folium.TileLayer(
        tiles='https://basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}.png',
        attr='©CartoDB',
        name='CartoDB Light No Labels'
    ).add_to(m)
    return m


def _filter_by_bbox(df, lat_col, lon_col, min_lat, max_lat, min_lon, max_lon):
    """Filter dataframe by bounding box coordinates."""
    return df[
        df[lat_col].between(min_lat, max_lat) &
        df[lon_col].between(min_lon, max_lon)
    ]


def _plot_segments(m, segment_df, colors=None):
    """Plot segments with random colors. Returns dict mapping segment_idx to color."""
    if colors is None:
        colors = MAP_COLORS
    
    segment_colors = {}
    for segment_idx, segment_row in segment_df.iterrows():
        color = random.choice(colors)
        segment_colors[segment_idx] = color
        
        segment_coords = [(lat, lon) for lon, lat in segment_row.geometry.coords]
        folium.PolyLine(
            locations=segment_coords,
            color=color,
            **SEGMENT_STYLE
        ).add_to(m)
    
    return segment_colors


def _plot_accidents_by_segment(m, joined_df, sliced_segments, segment_colors):
    """Plot accidents colored by their assigned segment. Returns lists of plotted coordinates."""
    lat_list = []
    lon_list = []
    
    for segment_idx in sliced_segments.index:
        color = segment_colors[segment_idx]
        segment_accidents = joined_df[joined_df["index_right"] == segment_idx]
        
        for _, row in segment_accidents.iterrows():
            lat = row['YGCSWGS84']
            lon = row['XGCSWGS84']
            lat_list.append(lat)
            lon_list.append(lon)
            
            folium.CircleMarker(
                location=[lat, lon],
                color=color,
                fill_color=color,
                **ACCIDENT_MARKER_STYLE
            ).add_to(m)
    
    return lat_list, lon_list


def _plot_unassigned_accidents(m, accident_locations_df, lat_list, lon_list, bbox):
    """Plot accidents not assigned to any segment in black."""
    min_lat, max_lat, min_lon, max_lon = bbox
    
    mask = (
        (~accident_locations_df['YGCSWGS84'].isin(lat_list) & 
         accident_locations_df['YGCSWGS84'].between(min_lat, max_lat)) & 
        (~accident_locations_df['XGCSWGS84'].isin(lon_list) & 
         accident_locations_df['XGCSWGS84'].between(min_lon, max_lon))
    )
    filtered_rows = accident_locations_df[mask]
    
    for _, row in filtered_rows.iterrows():
        folium.CircleMarker(
            location=[row['YGCSWGS84'], row['XGCSWGS84']],
            color="black",
            fill_color="black",
            **ACCIDENT_MARKER_STYLE
        ).add_to(m)


def _plot_crossings(m, crossing_dict, bbox):
    """Plot crossing points as large black markers."""
    min_lat, max_lat, min_lon, max_lon = bbox
    
    for lon, lat in crossing_dict.keys():
        if (min_lon < lon < max_lon) and (min_lat < lat < max_lat):
            folium.CircleMarker(
                location=[lat, lon],
                **CROSSING_MARKER_STYLE
            ).add_to(m)


def plot_segments_accidents(crossing_dict, segment_df, joined_df, accident_locations_df,
                            coordinates, lat_tol, lon_tol, zoom, save_path=None, save=False):
    """
    Plot segments, accidents, and crossings on an interactive map.
    
    Parameters:
    -----------
    crossing_dict : dict
        Dictionary mapping coordinates to crossing segments
    segment_df : GeoDataFrame
        Segment geometries with latitude/longitude columns
    joined_df : DataFrame
        Accidents joined to segments with index_right column
    accident_locations_df : DataFrame
        All accident locations with coordinate columns
    coordinates : tuple
        (lat, lon) center point for the map
    lat_tol : float
        Latitude tolerance for bounding box
    lon_tol : float
        Longitude tolerance for bounding box
    zoom : int
        Initial zoom level
    save_path : str, optional
        Path to save the map HTML
    save : bool, optional
        Whether to save the map
    
    Returns:
    --------
    folium.Map
        Interactive map object
    """
    # Calculate bounding box
    bbox = _calculate_bbox(coordinates, lat_tol, lon_tol)
    min_lat, max_lat, min_lon, max_lon = bbox
    
    # Create base map
    m = _create_base_map(coordinates, zoom)
    
    # Filter segments to bounding box
    sliced_segments = _filter_by_bbox(segment_df, "latitude", "longitude", *bbox)
    
    # Plot segments and get color mapping
    segment_colors = _plot_segments(m, sliced_segments)
    
    # Plot accidents by segment
    lat_list, lon_list = _plot_accidents_by_segment(m, joined_df, sliced_segments, segment_colors)
    
    # Plot unassigned accidents
    _plot_unassigned_accidents(m, accident_locations_df, lat_list, lon_list, bbox)
    
    # Plot crossings
    _plot_crossings(m, crossing_dict, bbox)
    
    # Save if requested
    if save and save_path:
        m.save(save_path)
        print(f"Map saved to {save_path}")
    
    return m


# Example usage
coordinates = (52.518589, 13.376665)  # Berlin Mitte
lat_tol = 1.5e-2
lon_tol = 3e-2
zoom = 14

map_obj = plot_segments_accidents(
    all_crossings, 
    strava_segments, 
    accidents_with_segment, 
    accident_locations,
    coordinates, 
    lat_tol, 
    lon_tol, 
    zoom, 
    save=False
)
map_obj